# 2. Extrahieren der benötigten Informationen zum Download der Objekte

### Laden benötigter Zusatzbibliotheken: 

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


## Inhalte aus dem XML extrahieren: 

### Datenset aus Datei laden: 

In [ ]:
tree = etree.parse('datadump.xml')

root = tree.getroot()                
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
records = root.findall('.//marc:record', namespaces=ns)
print("Gefundene Records:", len(records))

### Einblick: 

In [ ]:
testrecord = records[54]

In [ ]:
print(testrecord)

In [ ]:
testrecord_text = etree.tostring(testrecord, encoding="unicode")
print(testrecord_text)

In [ ]:
# Extrahieren des Inhalts des Controlfields "001"
controlfield_001 = testrecord.find(".//controlfield[@tag='001']")
print(controlfield_001)

In [ ]:
# Definieren des Namespaces: 
namespace = {'marc': 'http://www.loc.gov/MARC21/slim'}

In [ ]:
controlfield_001 = testrecord.find('.//marc:controlfield[@tag="001"]', namespaces=namespace)#.text
print(controlfield_001)

In [ ]:
datafield_245_a = testrecord.xpath('.//marc:datafield[@tag="245"]/marc:subfield[@code="a"]/text()', namespaces=namespace)
print(datafield_245_a)

In [ ]:
print(type(datafield_245_a))

### Erstellen einer Funktion, um Inhalte nach Feldern aus XML zu extrahieren: 

In [ ]:
def parse_record(record):
    
    record = etree.tostring(record, encoding="unicode").replace('\x98', '').replace('\x9c', '')
    #record = str(record).replace('\x98', '').replace('\x9c', '')
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", record))
    
       
    def extract_text(xpath_query):
        fields = xml.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_gnd = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='0']")
    year = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")
    links = extract_text("marc:datafield[@tag='856']/marc:subfield[@code='u']")

    return {"idn": idn, "titel": titel, "author": author, "gnd":author_gnd, "year": year, "links":links}
                                  

In [ ]:
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

## Speichern: 

In [ ]:
df.to_csv("Studentenbewegung_free_pdf.csv", index=False, encoding="utf-8")